## Importing some schtuff
### Using simpy for the simulation

In [1]:
import simpy
import random
import statistics

In [2]:
r = 0.68 # constant for males
drink_alcohol = 14.0 # 14 grams of alcohol per 'standard' size drink

def get_user_input():
    """ Get user inputs needed for the simulation """
    weight = float(input('Enter weight in pounds: '))
    weight *= 454.0 # convert to grams
    print(f'Weight is {weight} in grams')
    
    party_hours = int(input('Enter party time in hours: '))
    party_minutes = party_hours * 60
    print(f'Party will be {party_minutes} minutes long')
    
    return (weight, party_minutes)

In [3]:
def calculate_bac(total_drinks, current_time):
    """ Use the Widmark formula to calculate BAC (assuming male bc bachelor party) """
    total_alcohol = total_drinks * drink_alcohol
    bac_decay = current_time / 60 * 0.015
    bac = total_alcohol / (weight *  r) * 100 - bac_decay
    return bac

In [19]:
class Party:
    def __init__(self, env):
        self.env = env
        self.calvin = simpy.Resource(env, 1)
        self.drinks = 0
        self.things_done = 0
        
    def do_fun_thing(self, ft):
        temp_desc = ft.desc
        marker_count = ft.desc.count('?')
        for i in range(0, marker_count):
            temp_desc = temp_desc.replace('?', random.choice(bachelors), 1)
        print(temp_desc)
        yield self.env.timeout(ft.duration)
        self.drinks += ft.drinks
        bac = calculate_bac(self.drinks, self.env.now)
        print(f'    Current time is {self.env.now}')
        print(f'    You\'ve had {self.drinks} drinks')
        print(f'    Your current bac is {bac} ;)')
        self.things_done += 1

In [10]:
class FunThing:
    def __init__(self, duration, drinks, desc):
        self.duration = duration
        self.drinks = drinks
        self.desc = desc

In [6]:
def start_fun_thing(env, ft, party):
    with party.calvin.request() as request:
        yield request
        yield env.process(party.do_fun_thing(ft))

In [7]:
bachelors = [ 'Logan', 'Vishal', 'Nathan', 'Naveen', 'Cal', 'Preston', 'Trevor', 'Veijay' ]
fun_things = [
    FunThing(30, 0, '? challenges you to a round of Smash Bros'),
    FunThing(15, 0, 'You play Twister with ? and ?'),
    FunThing(5, 1, '? pours you a shot of Kraken. Bottoms up!'),
    FunThing(30, 3, 'The whole party plays Boom. You get the boom cup ¯\_(ツ)_/¯'),
    FunThing(15, 0, 'You play some Rocket League with ? and reach Grand Champion #dagger #obese #quitoutofrespect'),
    FunThing(10, 0, '? starts telling you about his favorite craft beer. You drink one to appease him'),
    FunThing(20, 2, 'You go to grab a beer from the fridge and get Iced by ?. Two for the price of one!'),
    FunThing(30, 1, 'You try a new beer with ? and it\'s awesome'),
    FunThing(5, 1, '? pours you a shot of Kraken. Bottoms up!'),
    FunThing(30, 1, 'The whole party plays Boom. ? gets the boom cup'),
]

In [8]:
def run_party(env):
    party = Party(env)
    for _ in range(3):
        ft = random.choice(fun_things)
        env.process(start_fun_thing(env, ft, party))
        
    while True:
        yield env.timeout(1)
        ft = random.choice(fun_things)
        env.process(start_fun_thing(env, ft, party))
        

In [21]:
weight, time_minutes = get_user_input()

Enter weight in pounds: 180
Weight is 81720.0 in grams
Enter party time in hours: 4
Party will be 240 minutes long


In [24]:
env = simpy.Environment()
env.process(run_party(env))
env.run(until=time_minutes)


Naveen starts telling you about his favorite craft beer. You drink one to appease him
    Current time is 10
    You've had 0 drinks
    Your current bac is -0.0024999999999999996 ;)
You play Twister with Preston and Veijay
    Current time is 25
    You've had 0 drinks
    Your current bac is -0.00625 ;)
The whole party plays Boom. Naveen gets the boom cup
    Current time is 55
    You've had 1 drinks
    Your current bac is 0.011443631050070544 ;)
Preston pours you a shot of Kraken. Bottoms up!
    Current time is 60
    You've had 2 drinks
    Your current bac is 0.035387262100141086 ;)
You go to grab a beer from the fridge and get Iced by Preston. Two for the price of one!
    Current time is 80
    You've had 4 drinks
    Your current bac is 0.08077452420028217 ;)
Preston pours you a shot of Kraken. Bottoms up!
    Current time is 85
    You've had 5 drinks
    Your current bac is 0.1047181552503527 ;)
Cal pours you a shot of Kraken. Bottoms up!
    Current time is 90
    You've 